## ReMKiT1D input generator - testing kinetic drag heating operator

This notebook tests whether the implemented common_models.py drag heating operator for the $f_0$ harmonic performs as expected.


In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_kin_heating_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 4 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

timeNorm = skn.collTimeei(rk.normalization["eVTemperature"],rk.normalization["density"],rk.normalization["referenceIonZ"])

### Grid setup

In [ ]:
xGrid = 0.1*np.ones(128) 
dv0 = 0.01
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0
gridObj = Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True,isPeriodic=True)
L = sum(xGrid)


In [ ]:
# Add the grid to the wrapper
rk.grid = gridObj

### Set default species

In [ ]:
rk.addSpecies("e",0)
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0)
rk.setStandardTextbookOptions([0]) 


### Variables

In [ ]:
n = 1.0*np.ones(gridObj.numX())# + 0.001*np.sin(2*np.pi*gridObj.xGrid/L)
T = 1
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (T*np.pi)**(-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T)

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = gridObj.velocityMoment(f,0,1)
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]
    
rk.addVar("f",f,isDistribution=True,isCommunicated=True)
rk.addVar("n",n,isDerived=True,units='$10^{19} m^{-3}$',derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("W",isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"]))
rk.addVar("zeroVar",isDerived=True,outputVar=False)
rk.addVar("T",isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","zeroVar"]))
rk.addVar("time",isScalar=True,isDerived=True)

rk.addCustomDerivation("momDeriv1",sc.momentDerivation(1,1))
rk.addVar("mom1f0",n,isDerived=True,derivationRule=sc.derivationRule("momDeriv1",["f"]))

### Adding the heating

In [ ]:
energyInjectionRate = 0.02
heatTerm = cm.dvEnergyTerm("f",sc.VarData(["n","mom1f0"],[1.0,-1.0]),rk,-energyInjectionRate,k=1)
heatModel = sc.CustomModel("heating")
heatModel.addTerm("dragHeating",heatTerm)
rk.addModel(heatModel.dict())

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0) #Everything default except for more lenient absolute convergence tolerance

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep =  0.1

rk.setIntegratorGlobalData(1,1,initialTimestep) 

In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(100)
rk.setFixedStepOutput(10)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 10

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore="zeroVar")
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Test distribution function distortion due to heating operator

In [ ]:
dataName = 'f'
xInd = 60 

times = loadedData.coords['time'].data
f0_analytic=np.zeros((numFiles+1,gridObj.numV()))
temps = loadedData["T"].data
dens = loadedData["n"].data
for i in range(numFiles+1):
        f0_analytic[i,:] = (temps[i,xInd]*np.pi)**(-1.5) * dens[i,xInd] * np.exp(-gridObj.vGrid**2/temps[i,xInd])
        
curveDict = {t: hv.Curve([(v**2,loadedData[dataName].data[t,xInd,0,i]) for i,v in enumerate(loadedData.coords["v"].data)]).opts(logy=True,ylim=(1e-20,1e0),xlim=(0,60))*
              hv.Curve([(v**2,f0_analytic[t,i]) for i,v in enumerate(loadedData.coords["v"].data)]).opts(logy=True,ylim=(1e-20,1e0),xlim=(0,60))for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check spatial profile

In [ ]:
curveDict = {t: hv.Curve(loadedData["W"][{"time":t}]) for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check heating amount 

The error is likely mostly due to velocity space integration

In [ ]:
heatingError = (loadedData["W"].data[1,63]-loadedData["W"].data[0,63] - 0.02)/0.02
heatingError

